# Interactive Plot

Load and listen to Data:

In [1]:
from scipy import signal
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import librosa
import IPython.display as ipd
import numpy as np
import parselmouth

from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColorBar, LogColorMapper, LogTicker, LinearAxis, Range1d


In [2]:
audio1, fs = librosa.load('a_8000.wav')
print("f_s =", fs)
ipd.Audio('a_8000.wav') # show audio player

f_s = 22050


In [9]:
snd = parselmouth.Sound("f116.wav")
windowLength = 2 #millisecond
minimumPitch = 1000/windowLength

intensity = snd.to_intensity(minimum_pitch=minimumPitch)

p = figure(title="Test",plot_width=600, plot_height=400, x_range=(snd.xs()[0], snd.xs()[-1]), y_range=(np.floor(np.min(snd.values)*10)/10, np.ceil(np.max(snd.values)*10)/10))
p.line(snd.xs(), np.squeeze(snd.values), line_width = 0.5, color = '#BEBEBE')
p.xaxis.axis_label = 't in s'
p.yaxis.axis_label = 'lin. amplitude'

p.extra_y_ranges = {"intensity": Range1d(start=np.floor(np.min(intensity.values)/10)*10, end=np.ceil(np.max(intensity.values)/10)*10)}
p.line(intensity.xs(), np.squeeze(intensity.values), line_width = 0.5, color = 'red', y_range_name="intensity")
p.add_layout(LinearAxis(y_range_name="intensity", axis_label='Intensity in dB'), 'right')

output_notebook()
show(p,notebook_handle=True)

Loading BokehJS ...

## Display Spectrogram of Data:

In [5]:
window='hann'
windowlength = 2048
overlap = windowlength-1

fVec, tVec, spectroData = signal.spectrogram(audio1, fs, window=window, nperseg=windowlength, noverlap=overlap)
frameVec = np.array(range(0, len(tVec)-1))

index = (np.abs(fVec - 1500)).argmin()
spectroData = np.delete(spectroData, range(index, len(spectroData)), 0)
fVec = np.delete(fVec, range(index, len(fVec)))


spectroData = np.divide(spectroData, np.max(spectroData))

In [47]:

timeWidget = widgets.FloatSlider(min=tVec[0], max=tVec[-1], step=tVec[1]-tVec[0], value=tVec[0], description="Time in s")
timeInStft = timeWidget.value
stftFrame = (np.abs(tVec - timeInStft)).argmin()

output_notebook()
p1 = figure(title="Spetrogram of Sound Sample",plot_width=600, plot_height=300, x_range=(tVec[0],tVec[-1]), y_range=(fVec[0],fVec[-1]))
p1.xaxis.axis_label = 'Time in s'
p1.yaxis.axis_label = 'Frequency in Hz'
color_mapper = LogColorMapper(palette="Viridis256", low=1e-3, high=1)
p1.image(image=[spectroData+np.abs(np.min(spectroData))], x=tVec[0], y=fVec[0], dw=tVec[-1], dh=fVec[-1], color_mapper=color_mapper)
p1.grid.visible=False
spectrumLine = p1.line([timeInStft, timeInStft], [fVec[0], fVec[-1]], line_width = 2, color = 'red')

p2 = figure(title="Spectrum of Selected STFT Frame",plot_width=600, plot_height=200, x_range=(fVec[0],fVec[-1]), y_range=(np.min(20*np.log10(spectroData[:,stftFrame])),0))
p2.xaxis.axis_label = 'Frequency in Hz'
p2.yaxis.axis_label = 'relative Magnitude in dB'
spectrumPlot = p2.line(fVec, np.log10(spectroData[:,stftFrame])*20, line_width = 2, color = 'red')

pAll = gridplot([[p1], [p2]])
show(pAll,notebook_handle=True)

def update_plot(timeInStft, stftFrame):
    spectrumLine.data_source.data['x'] = [timeInStft, timeInStft]
    spectrumPlot.data_source.data['y'] = 20*np.log10(spectroData[:,stftFrame])
    push_notebook()

def on_value_change(change):
    timeInStft = timeWidget.value
    stftFrame = (np.abs(tVec - timeInStft)).argmin()
    update_plot(timeInStft, stftFrame)

timeWidget.observe(on_value_change,names='value')

widgets.HBox([timeWidget])

NameError: name 'tVec' is not defined